In [3]:
import pandas as pd
import numpy as np
import cv2
import imutils
import json 
import requests

## Get face rectangles from each frame

In [4]:
# get the frontalface xml file
url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

response = requests.get(url)
# content = bytes(response.content)
# Load the XML file into memory as a string
xml_string = response.content.decode('utf-8')

# Create a FileStorage object and read the XML string into it
fs = cv2.FileStorage(xml_string, cv2.FILE_STORAGE_READ | cv2.FILE_STORAGE_MEMORY)
cascade = cv2.CascadeClassifier()
cascade.read(fs.getFirstTopLevelNode())

True

In [5]:
rows = []


# Function to extract frames
def FrameCapture(path):
  # Path to video file
  vid_obj = cv2.VideoCapture(path)

  # checks whether frames were extracted
  success = 1
  frame_count = 0

  while success:
      # vidObj object calls read
      # function extract frames
      success, image = vid_obj.read()

      if image is None: 
        break

      #loading face detector
      detector = cv2.CascadeClassifier()
      detector.read(fs.getFirstTopLevelNode())

      # resize image and convert it to grayscale
      image = imutils.resize(image, width=500)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      # detect faces in image using the haar cascade face detector
      rects = detector.detectMultiScale(image=gray, 
                                        scaleFactor=1.1, 
                                        minNeighbors=9, 
                                        minSize=(30, 30), 
                                        flags=cv2.CASCADE_SCALE_IMAGE)

      #extract x,y,w,h of each rect and store in dataframe
      for (x, y, w, h) in rects:
        row = {'frame': frame_count, 'x': x, 'y': y, 'w': w, 'h': h}
        rows.append(row)

      frame_count += 1
  return pd.DataFrame(rows)

In [6]:
video_path = './ML0001_1.mp4'
video_df = FrameCapture(video_path)

In [7]:
video_df.head(3)

,frame,x,y,w,h
0,0,196,351,150,150
1,1,194,350,153,153
2,2,192,349,155,155


## Fit the bounding boxes

In [8]:
video_path = './ML0001_1.mp4'
video_obj = cv2.VideoCapture(video_path)

for index, row in video_df.iterrows():
    # read the frame from the video
    video_obj.set(cv2.CAP_PROP_POS_FRAMES, row['frame'])
    ret, frame = video_obj.read()
    
    x, y, w, h = int(row['x']), int(row['y']), int(row['w']), int(row['h'])
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Frame with bounding box", frame)
    cv2.waitKey(1)

video.release()

KeyboardInterrupt: 

## Format in JSON

In [ ]:
json_dict = {
    'input_file_name': path,
    'input_resolution': '',
    'generator': 'facerectangles.exe',
    'date': '',
    'frame_count': frame_count,
    'output': video_df.to_json(orient = 'frame')
}

In [ ]:
json_dict = {
    
}

In [ ]:
output_path = path[0:-4] + '_facerectanges.json'
json_object = json.dump(json_dict, output_path, indent = 4) 